feature_list:  
 - 100000 vectors
 - 512 dimension  
  
Goal:  
find a library that calculates cosine_similarity faster

In [1]:
import pickle
import numpy as np
import math
import time, timeit

In [2]:
with open('feature_list.txt', "rb") as fp:   
        data_dict = pickle.load(fp)

In [3]:
values_list = []
for i in range(len(data_dict)):
    for k, v in data_dict[i].items():
        values_list.append(v)

In [4]:
value_1 = values_list[0]

In [5]:
num_times = 100

In [6]:
def new_list(buf_list, idx):
    return buf_list[1:idx+1]

In [7]:
def avg_t(t, num):
        return np.mean(t) / num

In [8]:
def format_t(t):
    """Turn t into nice formating"""
    if t < 1e-3:
        return "{:.3f} ns".format(t * 1e6)
    elif t < 1:
        return "{:.3f} ms".format(t * 1e3)
    else:
        return "{:.2f} s".format(t)

In [9]:
cases = [300, 1000, 10000, 99999]

In [10]:
values_300 = new_list(values_list, 300)
values_1000 = new_list(values_list, 1000)
values_10000 = new_list(values_list, 10000)
values_99999 = new_list(values_list, 99999)

# Numpy

In [11]:
results = []
total_time = 0
for i in range(num_times):
    start_time = time.time()
    for item in values_300:
        cosine = np.dot(value_1, item)
        theta = np.arccos(cosine)
        theta = theta * 180 / np.pi
        results.append(theta)
    total_time += (time.time() - start_time)
print('time = {}'.format(format_t(avg_t(total_time, num_times))))

time = 1.297 ms


In [12]:
def calc_numpy(buf_vec, buf_data, buf_times):
    total_time = 0
    for i in range(buf_times):
        results = []
        start_time = time.time()
        for item in buf_data:
            cosine = np.dot(buf_vec, item)
            theta = np.arccos(cosine)
            theta = theta * 180 / np.pi
            results.append(theta)
        total_time += (time.time() - start_time)
    return total_time

In [13]:
vecs_list = [values_300, values_1000, values_10000, values_99999]
# vecs_list = [values_300, values_1000]
for item in list(zip(cases,vecs_list)):
    t = calc_numpy(value_1, item[1], num_times)
    print('Calculation time for {} = {}'.format(item[0], format_t(avg_t(t, num_times))))

Calculation time for 300 = 1.252 ms
Calculation time for 1000 = 4.059 ms
Calculation time for 10000 = 41.350 ms
Calculation time for 99999 = 411.746 ms


# Numpy + math

In [14]:
def calc_numpy_math(buf_vec, buf_data, buf_times):
    total_time = 0
    for i in range(buf_times):
        results = []
        start_time = time.time()
        for item in buf_data:
            cosine = np.dot(buf_vec, item)
            theta = math.acos(cosine)
            theta = theta * 180 / math.pi
            results.append(theta)
        total_time += (time.time() - start_time)
    return total_time

In [15]:
for item in list(zip(cases,vecs_list)):
    t = calc_numpy_math(value_1, item[1], num_times)
    print('Calculation time for {} = {}'.format(item[0], format_t(avg_t(t, num_times))))

Calculation time for 300 = 329.142 ns
Calculation time for 1000 = 1.087 ms
Calculation time for 10000 = 11.336 ms
Calculation time for 99999 = 116.032 ms


# Torch

In [17]:
import torch

In [18]:
import torch.nn.functional as F

In [19]:
torch.cuda.is_available()

True

In [20]:
torch.cuda.get_device_name(0)

'GeForce RTX 2080'

In [21]:
tensor_value_1 = torch.from_numpy(value_1).cuda()

### CPU -> GPU -> CPU

In [22]:
def calc_torch_tr__(buf_vec, buf_data, buf_times):
    total_time = 0
    for i in range(buf_times):
        results = []
        step_time = 0
        for j in range(len(buf_data)):
            t0 = time.time()
            item_torch = torch.from_numpy(buf_data[j]).cuda()
            output = F.cosine_similarity(tensor_value_1, item_torch, dim=-1)
            output = output.cpu().numpy()
            results.append(output)
            if j > 0:
                step_time += time.time() - t0
            #print('{:.7f}'.format(step_time))
        #print('#################################')
        total_time += step_time
    return total_time

In [23]:
torch_vecs_list = [values_300, values_1000]

In [24]:
for item in list(zip(cases,torch_vecs_list)):
    t = calc_torch_tr__(tensor_value_1, item[1], num_times)
    print('Calculation time for {} = {}'.format(item[0], format_t(avg_t(t, num_times))))

Calculation time for 300 = 118.566 ms
Calculation time for 1000 = 381.944 ms


### GPU

In [25]:
list_tensor_300 = [torch.from_numpy(item).cuda() for item in values_300]
list_tensor_1000 = [torch.from_numpy(item).cuda() for item in values_1000]
list_tensor_10000 = [torch.from_numpy(item).cuda() for item in values_10000]
list_tensor_99999 = [torch.from_numpy(item).cuda() for item in values_99999]
# torch_list = [list_tensor_300, list_tensor_1000, list_tensor_10000, list_tensor_99999]
torch_list = [list_tensor_300, list_tensor_1000]

In [26]:
def calc_torch(buf_vec, buf_data, buf_times):
    total_time = 0
    for i in range(buf_times):
        #start_time = time.time()
        step_time = 0
        for item in buf_data:
            t0 = time.time()
            output = F.cosine_similarity(buf_vec, item, dim=-1)
            step_time += time.time() - t0
            #print('{:.7f}'.format(step_time))
        #total_time += (time.time() - start_time)
        #print('#################################')
        total_time += step_time
    return total_time

In [27]:
for item in list(zip(cases,torch_list)):
    t = calc_torch(tensor_value_1, item[1], num_times)
    print('Calculation time for {} = {}'.format(item[0], format_t(avg_t(t, num_times))))

Calculation time for 300 = 80.085 ms
Calculation time for 1000 = 273.835 ms


# Sklearn

In [28]:
from sklearn.metrics.pairwise import cosine_similarity

In [29]:
sklearn_value_1 = value_1.reshape(1,-1)

In [30]:
list_sklearn_300 = [item.reshape(1,-1) for item in values_300]
list_sklearn_1000 = [item.reshape(1,-1) for item in values_1000]
list_sklearn_10000 = [item.reshape(1,-1) for item in values_10000]
list_sklearn_99999 = [item.reshape(1,-1) for item in values_99999]
# sklearn_list = [list_sklearn_300, list_sklearn_1000, list_sklearn_10000, list_sklearn_99999]
sklearn_list = [list_sklearn_300, list_sklearn_1000]

In [31]:
def calc_sklearn(buf_vec, buf_data, buf_times):
    total_time = 0
    for i in range(buf_times):
        start_time = time.time()
        for item in buf_data:
            cosine = cosine_similarity(buf_vec, item)
        total_time += (time.time() - start_time)
    return total_time

In [32]:
for item in list(zip(cases,sklearn_list)):
    t = calc_sklearn(sklearn_value_1, item[1], num_times)
    print('Calculation time for {} = {}'.format(item[0], format_t(avg_t(t, num_times))))

Calculation time for 300 = 39.014 ms
Calculation time for 1000 = 132.089 ms


# Numba

In [33]:
import numba
from numba import jit, guvectorize, float32, cuda

### @jit

In [34]:
@jit(fastmath=True)
def calc_cosine(a, b):
    cosine = np.dot(a, b)
    theta = math.acos(cosine)
    theta = theta * 180 / math.pi
    
    return theta

In [35]:
for item in list(zip(cases,vecs_list)):
    t = 0
    for i in range(num_times):
        results = []
        start_time = time.time()
        for vec in item[1]:
            output = calc_cosine(value_1, vec)
            results.append(output)
        t += (time.time() - start_time)
    print('Calculation time for {} = {}'.format(item[0], format_t(avg_t(t, num_times))))

Calculation time for 300 = 2.883 ms
Calculation time for 1000 = 538.542 ns
Calculation time for 10000 = 6.044 ms
Calculation time for 99999 = 60.673 ms


### guvectorize

In [36]:
@guvectorize(['void(float32[:],float32[:], float32)'], '(n),(n)->()')
def calc_cosine_guv(a, b, theta):
    cosine = np.dot(a, b)
    theta = math.acos(cosine)
    theta = theta * 180 / math.pi

<ipython-input-36-797180d4863b>:3: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float32, 1d, A), array(float32, 1d, A))
  cosine = np.dot(a, b)


In [37]:
for item in list(zip(cases,vecs_list)):
    t = 0
    for i in range(num_times):
        results = []
        start_time = time.time()
        for vec in item[1]:
            output = calc_cosine_guv(value_1, vec)
            results.append(output)
        t += (time.time() - start_time)
    print('Calculation time for {} = {}'.format(item[0], format_t(avg_t(t, num_times))))

Calculation time for 300 = 405.512 ns
Calculation time for 1000 = 1.267 ms
Calculation time for 10000 = 13.491 ms
Calculation time for 99999 = 134.331 ms


### Cuda

In [38]:
@cuda.jit('void(float32[:],float32[:], float32[:])')
def mult_gpu_1d(a, b, c):
    x = cuda.grid(1)
    c[0] = 0
    if x < a.size:
        c[0] += a[x] * b[x]
    c[0] = math.acos(c[0]) * 180 / math.pi

In [39]:
value_1_cuda = cuda.to_device(value_1)

In [40]:
new_vecs_list = [values_300, values_1000]

In [41]:
for item in list(zip(cases,new_vecs_list)):
    t = 0
    for i in range(num_times):
        results = []
        result = cuda.device_array(shape=(1,), dtype=np.float32)
        start_time = time.time()
        for vec in item[1]:
            vec_cuda = cuda.to_device(vec)
            mult_gpu_1d(value_1_cuda, vec_cuda, result)
            results.append(result.copy_to_host())
        t += (time.time() - start_time)
    print('Calculation time for {} = {}'.format(item[0], format_t(avg_t(t, num_times))))

Calculation time for 300 = 120.586 ms
Calculation time for 1000 = 409.624 ms


In [42]:
list_cuda_300 = [cuda.to_device(item) for item in values_300]
list_cuda_1000 = [torch.from_numpy(item).cuda() for item in values_1000]
list_cuda_10000 = [torch.from_numpy(item).cuda() for item in values_10000]
list_cuda_99999 = [torch.from_numpy(item).cuda() for item in values_99999]
# cuda_list = [list_cuda_300, list_cuda_1000, list_cuda_10000, list_cuda_99999]
cuda_list = [list_cuda_300, list_cuda_1000]

In [43]:
for item in list(zip(cases,cuda_list)):
    t = 0
    for i in range(num_times):
        results = []
        result = cuda.device_array(shape=(1,), dtype=np.float32)
        start_time = time.time()
        for vec in item[1]:
            mult_gpu_1d(value_1_cuda, vec, result)
        t += (time.time() - start_time)
    print('Calculation time for {} = {}'.format(item[0], format_t(avg_t(t, num_times))))

Calculation time for 300 = 14.701 ms
Calculation time for 1000 = 108.889 ms


Calculation time for 300 = 20.068 ms  
Calculation time for 1000 = 161.570 ms  
Calculation time for 10000 = 1.58 s  
Calculation time for 99999 = 15.82 s  

In [ ]:
TPB = 32

@cuda.jit
def fast_matmul(A, B, C):
    """
    Perform matrix multiplication of C = A * B
    Each thread computes one element of the result matrix C
    """

    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, ), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, ), dtype=float32)

    x = cuda.grid(1)
    
    tx = cuda.threadIdx.x
    
    if x >= C.shape[0]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0
    for i in range(int(A.shape[0] / TPB)):
        # Preload data into shared memory
        sA[tx] = A[tx + i * TPB]
        sB[tx] = B[tx + i * TPB]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx] * sB[tx]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[0] = tmp
    
    C[0] = math.acos(C[0]) * 180 / math.pi

In [ ]:
threadsperblock = (TPB, )
blockspergrid_x = int(math.ceil(value_1.shape[0] / threadsperblock[0]))
blockspergrid = (blockspergrid_x, )

In [ ]:
for item in list(zip(cases,vecs_list)):
    t = 0
    for i in range(num_times):
        results = []
        result = cuda.device_array(shape=(1,), dtype=np.float32)
        start_time = time.time()
        for vec in item[1]:
            vec_cuda = cuda.to_device(vec)
            fast_matmul[blockspergrid, threadsperblock](value_1_cuda, vec, result)
            results.append(result.copy_to_host())
        t += (time.time() - start_time)
    print('Calculation time for {} = {}'.format(item[0], format_t(avg_t(t, num_times))))

In [ ]:
for item in list(zip(cases,cuda_list)):
    t = 0
    for i in range(num_times):
        results = []
        result = cuda.device_array(shape=(1,), dtype=np.float32)
        start_time = time.time()
        for vec in item[1]:
            #result = cuda.device_array(shape=(1,), dtype=np.float32)
            fast_matmul[blockspergrid, threadsperblock](value_1_cuda, vec, result)
        t += (time.time() - start_time)
    print('Calculation time for {} = {}'.format(item[0], format_t(avg_t(t, num_times))))

### Cupy

In [44]:
import cupy as cp

In [45]:
value_1_cupy = cp.asarray(value_1)

In [46]:
def calc_cupy(buf_vec, buf_data, buf_times):
    total_time = 0
    for i in range(buf_times):
        results = []
        start_time = time.time()
        for item in buf_data:
            item_cupy = cp.asarray(item)
            cosine = cp.dot(buf_vec, item_cupy)
            theta = cp.arccos(cosine)
            theta = theta * 180 / np.pi
            results.append(theta.get())
        total_time += (time.time() - start_time)
    return total_time

In [48]:
cupy_vecs_list = [values_300, values_1000]

In [49]:
for item in list(zip(cases,cupy_vecs_list)):
    t = calc_cupy(value_1_cupy, item[1], num_times)
    print('Calculation time for {} = {}'.format(item[0], format_t(avg_t(t, num_times))))

Calculation time for 300 = 56.801 ms
Calculation time for 1000 = 192.784 ms


In [50]:
list_cupy_300 = [cp.asarray(item) for item in values_300]
list_cupy_1000 = [cp.asarray(item) for item in values_1000]
list_cupy_10000 = [cp.asarray(item) for item in values_10000]
list_cupy_99999 = [cp.asarray(item) for item in values_99999]
# cupy_list = [list_cupy_300, list_cupy_1000, list_cupy_10000, list_cupy_99999]
cupy_list = [list_cupy_300, list_cupy_1000]

In [51]:
def calc_cupy_(buf_vec, buf_data, buf_times):
    total_time = 0
    for i in range(buf_times):
        results = []
        start_time = time.time()
        for item in buf_data:
            with cp.cuda.Device(0):
                cosine = cp.dot(buf_vec, item)
                theta = cp.arccos(cosine)
                theta = theta * 180 / np.pi
        total_time += (time.time() - start_time)
    return total_time

In [52]:
for item in list(zip(cases,cupy_list)):
    t = calc_cupy_(value_1_cupy, item[1], num_times)
    print('Calculation time for {} = {}'.format(item[0], format_t(avg_t(t, num_times))))

Calculation time for 300 = 28.828 ms
Calculation time for 1000 = 95.805 ms


In [53]:
def calc_cupy__(buf_vec, buf_data, buf_times):
    total_time = 0
    for i in range(buf_times):
        results = []
        step_time = 0
        for item in buf_data:
            item_cupy = cp.asarray(item)
            t0 = time.time()
            cosine = cp.dot(buf_vec, item_cupy)
            theta = cp.arccos(cosine)
            theta = theta * 180 / np.pi
            step_time += time.time() - t0
            results.append(theta.get())
        total_time += step_time
    return total_time

In [54]:
for item in list(zip(cases,cupy_vecs_list)):
    t = calc_cupy__(value_1_cupy, item[1], num_times)
    print('Calculation time for {} = {}'.format(item[0], format_t(avg_t(t, num_times))))

Calculation time for 300 = 28.446 ms
Calculation time for 1000 = 95.751 ms
